<a href="https://colab.research.google.com/github/ysjgithub/dl/blob/master/alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

plt.ion()   # interactive mode

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32,padding=4),
                       transforms.ToTensor()
                   ])), batch_size=128, shuffle=True, num_workers=4)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
    ])), batch_size=128, shuffle=True, num_workers=4)




Extracting ./cifar-10-python.tar.gz to .


In [0]:

model= nn.Sequential(
    nn.Conv2d(3,96,kernel_size=3,padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
    nn.Conv2d(96,256,kernel_size=3,padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
    nn.Conv2d(256, 384, kernel_size=3,padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 384, kernel_size=3,padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=3,padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
    nn.Flatten(),
    nn.Linear(4096,4096),
    nn.Linear(4096,10),
    nn.Softmax()
).to(device)
summary(model, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 34, 34]           2,688
              ReLU-2           [-1, 96, 34, 34]               0
         MaxPool2d-3           [-1, 96, 17, 17]               0
            Conv2d-4          [-1, 256, 19, 19]         221,440
              ReLU-5          [-1, 256, 19, 19]               0
         MaxPool2d-6            [-1, 256, 9, 9]               0
            Conv2d-7            [-1, 384, 9, 9]         885,120
              ReLU-8            [-1, 384, 9, 9]               0
            Conv2d-9            [-1, 384, 9, 9]       1,327,488
             ReLU-10            [-1, 384, 9, 9]               0
           Conv2d-11            [-1, 256, 9, 9]         884,992
             ReLU-12            [-1, 256, 9, 9]               0
        MaxPool2d-13            [-1, 256, 4, 4]               0
          Flatten-14                 [-

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [0]:

def one_hot(x, K):
    return np.array(x[:, None] == np.arange(K)[None, :], dtype=int)

def accuracy(data_loader):
    total_correct = 0
    for x, y in data_loader:
      x = x.to(device)
      y = one_hot(np.array(y.numpy()), 10)
      target_class = np.argmax(y, axis=1)
      predicted_class = np.argmax(model(x).cpu().detach().numpy(), axis=1)
      total_correct += np.sum(predicted_class == target_class)
    return total_correct / len(data_loader.dataset)
# accuracy(train_loader)


In [0]:
optimizer = optim.Adam(model.parameters(),weight_decay=0.001)

criterion = nn.CrossEntropyLoss().to(device)


In [0]:
root = "./drive/My Drive/experiment/CIFRA10/alexnet.pth"

torch.save(model.state_dict(),root)


In [0]:


def train(epoch):
    for x, y in train_loader:
      optimizer.zero_grad()
      x, y = x.to(device), y.to(device)
      output = model(x)
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
    train_accuracy_rate = accuracy(train_loader)
    test_accuracy_rate = accuracy(test_loader)
    print('epoch:{:04d}|loss:{:.4f}|'
      'train_accuracy:{:.4f}|test_accuracy:{:.4f}'.format(epoch,loss,train_accuracy_rate,test_accuracy_rate))
#   
# A simple test procedure to measure STN the performances on MNIST.
#
# train(1)

In [0]:
for i in range(200):
  train(i)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch:0000|loss:1.6999|train_accuracy:0.7588|test_accuracy:0.7486
epoch:0001|loss:1.7635|train_accuracy:0.7544|test_accuracy:0.7313
epoch:0002|loss:1.6401|train_accuracy:0.7567|test_accuracy:0.7414
epoch:0003|loss:1.7314|train_accuracy:0.7623|test_accuracy:0.7445
epoch:0004|loss:1.7604|train_accuracy:0.7576|test_accuracy:0.7375
epoch:0005|loss:1.7155|train_accuracy:0.7528|test_accuracy:0.7316
epoch:0006|loss:1.7019|train_accuracy:0.7510|test_accuracy:0.7290
epoch:0007|loss:1.7044|train_accuracy:0.7382|test_accuracy:0.7266
epoch:0008|loss:1.8105|train_accuracy:0.7238|test_accuracy:0.7000
epoch:0009|loss:1.6888|train_accuracy:0.7636|test_accuracy:0.7428
epoch:0010|loss:1.6507|train_accuracy:0.7525|test_accuracy:0.7449
epoch:0011|loss:1.7956|train_accuracy:0.7496|test_accuracy:0.7302
epoch:0012|loss:1.6351|train_accuracy:0.7635|test_accuracy:0.7441
epoch:0013|loss:1.6163|train_accuracy:0.7488|test_accuracy:0.7257
epoch:0014|loss:1.8308|train_accuracy:0.7491|test_accuracy:0.7366
epoch:0015